In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\zeyne\\Containerized-Bank-Customer-Churn-Prediction'

In [4]:
import pandas as pd

In [9]:
df = pd.read_csv("artifacts\data_ingestion\df.zip")
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
df.isnull().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [12]:
df.shape

(10000, 12)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [16]:
df.describe(include='object')

,country,gender
count,10000,10000
unique,3,2
top,France,Male
freq,5014,5457


In [17]:
df['churn'].value_counts()

churn
0    7963
1    2037
Name: count, dtype: int64

In [20]:
from scipy.stats import zscore

In [21]:
def outlier(df,threshold):
    num_cols= df.select_dtypes(include=['int64','float64'])
    z_score = num_cols.apply(zscore)

    return (abs(z_score > threshold)).sum().sort_values(ascending=False)


In [22]:
outlier(df,3)

age                 133
products_number      60
customer_id           0
credit_score          0
tenure                0
balance               0
credit_card           0
active_member         0
estimated_salary      0
churn                 0
dtype: int64

In [24]:
df.columns

Index(['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure',
       'balance', 'products_number', 'credit_card', 'active_member',
       'estimated_salary', 'churn'],
      dtype='object')

In [23]:
from dataclasses import dataclass
from pathlib import Path

In [25]:
@dataclass (frozen = True)
class DataValidationConfig :
    root_dir : Path
    status_file :str
    unzip_data_dir : Path
    all_schema : dict


In [26]:
from src.Containerized_Bank_Customed_Churn_Prediction.constants import *
from src.Containerized_Bank_Customed_Churn_Prediction.utils.common import read_yaml,create_directories

In [43]:
class ConfigurationManager:
 def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):

    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])


 def get_data_validation_config(self) -> DataValidationConfig:
    config = self.config.data_validation
    schema = self.schema.COLUMNS

    create_directories([config.root_dir])

    data_validation_config = DataValidationConfig(
        root_dir = config.root_dir,
        status_file = config.status_file,
        unzip_data_dir =config.unzip_dir,
        all_schema = schema,
    )

    return data_validation_config



In [29]:
import os
from Containerized_Bank_Customed_Churn_Prediction import logger


In [46]:
class DataValidation:
    def __init__(self, config:DataValidationConfig):
     self.config = config

    def validate_all_columns(self):
       
        try:
             validation_status : None
             df = pd.read_csv(self.config.unzip_data_dir)
             all_col = list(df.columns)
            
             all_schema = self.config.all_schema.keys()

             for cols in all_col:
              if cols not in all_schema:
               validation_status =False
               with open(self.config.status_file, 'w') as f:
                f.write(f" {validation_status}")
             
             else:
                validation_status = True
                with open(self.config.status_file,'w') as f:
                    f.write(f"{validation_status}")
            
             return validation_status

        except Exception as e:
            logger.info(e)
            raise e
             




In [47]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    logger.info(e)
    raise e

[2025-08-05 17:47:29,152:INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-08-05 17:47:29,154:INFO: common: yaml file:params.yaml loaded successfully]
[2025-08-05 17:47:29,157:INFO: common: yaml file:schema.yaml loaded successfully]
[2025-08-05 17:47:29,158:INFO: common: created directory at:artifacts]
[2025-08-05 17:47:29,160:INFO: common: created directory at:artifacts/data_validation]
